In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

In [2]:
pd.options.display.max_colwidth=100
np.set_printoptions(linewidth=140,edgeitems=10)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
import json
json_lines=[]

with open("tpc-dataset.train.txt",'r') as f:
    for line in f:
        json_lines.append(json.loads(line))

In [4]:
len(json_lines)

8607

In [5]:
train_df=pd.DataFrame.from_dict(json_lines)

In [6]:
train_df.head(3)

,age,education,id,texts
0,35-44,NaN,324781705,"[Уррааа!!!! А боялись, что шансов мало!! Уррааааа!!!!!!!)))))), Для стремянного или как там этот..."
1,25-34,NaN,971569377,"[Это для улицы ловушка) по периметру наставить ловушек- комаров должно быть меньше., Анна, аж зи..."
2,35-44,NaN,904248160,"[Вячеслав, смотри под видео на ютубе там есть ссылки на готовые контролеры и саму ленту, Телефон..."


In [72]:
Z=train_df[5000:8000]

In [73]:
Z.education.unique()

array(['lower', 'high', 'middle'], dtype=object)

In [8]:
Z.loc[:,'q'] = pd.Series(np.random.randn(len(Z)), index=Z.index)
for index, row in Z.iterrows():
    Z.at[index, 'q'] =len(Z.at[index,'texts'])
    Z.at[index,'text']=''.join(Z.at[index,'texts'])

/Users/macbook/.virtualenvs/py3/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/macbook/.virtualenvs/py3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
for index, row in Z.iterrows():
    Z.at[index, 'text']= Z['text'][index].lower().replace('.', ' ').replace(',', ' ').replace(';', ' ').replace(':', ' ').replace('!', ' ').replace('?', ' ').replace('\r', ' ').replace('\n', ' ')

In [10]:
Z.rename(columns={'age':'y'}, inplace=True)
Z.y

/Users/macbook/.virtualenvs/py3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


6000    18-24
6001    25-34
6002    25-34
6003    18-24
6004    18-24
6005     <=17
6006     <=17
6007    18-24
6008    35-44
6009    25-34
6010    25-34
6011     >=45
6012    18-24
6013     <=17
6014     <=17
6015     <=17
6016    25-34
6017     <=17
6018    18-24
6019    25-34
6020     <=17
6021    18-24
6022    25-34
6023     <=17
6024     <=17
6025    25-34
6026    25-34
6027     <=17
6028    18-24
6029    18-24
6030    35-44
6031    25-34
6032    25-34
6033     <=17
6034    18-24
6035     <=17
6036     <=17
6037     >=45
6038    25-34
6039     <=17
6040    25-34
6041     >=45
6042     <=17
6043     <=17
6044     <=17
6045    18-24
6046    25-34
6047     >=45
6048     <=17
6049     <=17
6050     <=17
6051     <=17
6052    25-34
6053    25-34
6054     <=17
6055    18-24
6056    25-34
6057     <=17
6058     >=45
6059    18-24
6060     <=17
6061    18-24
6062    25-34
6063     >=45
6064     <=17
6065     <=17
6066     <=17
6067     <=17
6068    18-24
6069    35-44
6070     >=45
6071  

In [11]:
cleanup_nums = { "y":       {"<=17": 1, "18-24": 2, "25-34":3, "35-44":4, ">=45":5},
                 "education": {"low": 1, "middle": 2,"high": 3} }
Z.replace(cleanup_nums, inplace=True)
#Z.education = Z.education.fillna(0)

/Users/macbook/.virtualenvs/py3/lib/python3.6/site-packages/pandas/core/generic.py:4550: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


In [13]:
Z.head(3)

,y,education,id,texts,q,text
6000,2.0,2,410290383,"[🍕🍱, , , , , , , Видимо игра затянется еще на сутки 😄, , , , , , 😴, !!!, , , , 😈, , , , Давайте ...",46.0,🍕🍱видимо игра затянется еще на сутки 😄😴 😈давайте уже столик пополам распилим 😂😂сейчас все высп...
6001,3.0,3,374138912,"[Есть два варианта:1 можно в суп.2 отпустить на все 4 стороны😏😅, Таюсик, Вам сюда WWF😏, Таюсик, ...",59.0,есть два варианта 1 можно в суп 2 отпустить на все 4 стороны😏😅таюсик вам сюда wwf😏таюсик если ...
6002,3.0,3,776479390,"[У меня в топ3, цвет 354, Смотрел на днях haibo smart. Были варианты в районе 1800, а сегодня 21...",118.0,у меня в топ3 цвет 354смотрел на днях haibo smart были варианты в районе 1800 а сегодня 2190 ...


In [66]:
train_sels = ~np.isnan(Z.y)
test_sels = np.isnan(Z.y)

In [67]:
print(train_sels)
print(test_sels)

6000     True
6001     True
6002     True
6003     True
6004     True
6005     True
6006     True
6007     True
6008     True
6009     True
6010     True
6011     True
6012     True
6013     True
6014     True
6015     True
6016     True
6017     True
6018     True
6019     True
6020     True
6021     True
6022     True
6023     True
6024     True
6025     True
6026     True
6027     True
6028     True
6029     True
6030     True
6031     True
6032     True
6033     True
6034     True
6035     True
6036     True
6037     True
6038     True
6039     True
6040     True
6041     True
6042     True
6043     True
6044     True
6045     True
6046     True
6047     True
6048     True
6049     True
6050     True
6051     True
6052     True
6053     True
6054     True
6055     True
6056     True
6057     True
6058     True
6059     True
6060     True
6061     True
6062     True
6063     True
6064     True
6065     True
6066     True
6067     True
6068     True
6069     True
6070     True
6071  

In [68]:
print(train_inds)
print(test_inds)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [69]:
train_inds = [i for i, x in enumerate(train_sels) if x]
train_inds

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [70]:
test_inds = [i for i, x in enumerate(test_sels) if x]
test_inds

[376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454,
 455,
 456,
 457,
 458,
 459,
 460,
 461,
 462,
 463,
 464,
 465,
 466,
 467,
 468,
 469,
 470,
 471,
 472,
 473,
 474,
 475,
 476,
 477,
 478,
 479,
 480,
 481,
 482,
 483,
 484,
 485,
 486,
 487,
 488,
 489,
 490,
 491,
 492,
 493,
 494,
 495,
 496,
 497,
 498,
 499,
 500,
 501,
 502,
 503,
 504,
 505,
 506,
 507,
 508,
 509,
 510,
 511,
 512,
 513,
 514,
 515,
 516,
 517,
 518,
 519,
 520,
 521,
 522,
 523,
 524,
 525,
 526,
 527,
 528,
 529,
 530,
 531,
 532,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 540,
 541,
 542

In [71]:
del train_sels, test_sels

In [63]:
y_train=Z.y.values[train_inds]

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [34]:
from sklearn.feature_extraction.text import  CountVectorizer


In [35]:
def extract_features_cv(train_texts, test_texts, ngrams_count, stp_wrds=None, mindf=2, maxdf=0.9):
    
    tfidf = CountVectorizer(ngram_range=(1, ngrams_count), stop_words=stp_wrds, min_df=mindf, max_df=maxdf)
    train_features = tfidf.fit_transform(train_texts)
    test_features = tfidf.transform(test_texts)
    return train_features, test_features

In [36]:
tit_train_features_cv, tit_test_features_cv = extract_features_cv(Z["text"].values[train_inds],
                                                                  Z["text"].values[test_inds], 2, 'english', 5, 0.9)

In [37]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
text_train_f = vectorizer.fit_transform(Z["text"].values[train_inds] )
text_test_f = vectorizer.transform(Z["text"].values[test_inds])

In [38]:
train_f = sp.hstack((text_train_f,tit_train_features_cv), format='csr')

In [39]:
test_f = sp.hstack((text_test_f,tit_test_features_cv), format='csr')

In [40]:
test_f.shape

(1624, 304061)

In [41]:
train_f.shape

(376, 304061)

In [42]:
clf = LogisticRegression(C=1.0)

In [43]:
clf.fit(train_f, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [44]:
Y_pred = clf.predict(test_f)

In [45]:
Y = pd.DataFrame(Y_pred)

In [46]:
Y

,0
0,3.0
1,1.0
2,1.0
3,3.0
4,3.0
5,1.0
6,2.0
7,3.0
8,3.0
9,2.0


In [47]:
cleanup_nums = {"age": {1.0: "<=17", 2.0: "18-24", 3.0: "25-34", 4.0: "35-44", 5.0: ">=45"}}
Y.columns=['age']
Y.replace(cleanup_nums, inplace=True)

In [48]:
Y

,age
0,25-34
1,<=17
2,<=17
3,25-34
4,25-34
5,<=17
6,18-24
7,25-34
8,25-34
9,18-24


In [49]:
Y.to_csv('sol2.csv',index=False)

In [50]:
Y.at[0,"age"]


'25-34'

# test

In [51]:
texts=["hi", "ejwj"]

In [52]:
df = pd.DataFrame({"q": len(texts), "text": "".join(texts), }, index=[0])
tit_train_features_cv, tit_test_features_cv = extract_features_cv(Z["text"].values[train_inds],
                                                                  df['text'].values, 2, 'english', 5, 0.9)




In [53]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
text_train_f = vectorizer.fit_transform(Z["text"].values[train_inds] )
text_test_f = vectorizer.transform(df["text"].values)

In [54]:
test_f = sp.hstack((text_test_f, tit_test_features_cv), format='csr')



In [55]:
df_t = clf.predict(test_f)

In [56]:
df_t[0]

1.0